In [1]:
import pandas as pd
import xarray as xr
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

% matplotlib inline

# Read cleaned data

In [25]:
disaster_summaries = pd.read_csv(
    'DataVizDisasterSummariesFV12.19.2016.explicitly_named_hurricanes.csv',
    skipinitialspace=True)

In [26]:
disaster_summaries.head()

,Disaster Number,Year,Region,Region Number,State Abbreviation,State,County,Declaration Date,Disaster Type,Incident Type,...,Incident End Date,Disaster Close Out Date,fips,state_fips,county_fips,school_district,declaration_date,ibtracs_storm_id,ibtracs_storm_name,ibtracs_storm_season
0,107,1960,Region IV,4,NC,North Carolina,Statewide,9/16/60,DR,Hurricane,...,9/16/60,3/30/62,37000.0,37,NaN,NaN,1960-09-16 18:00:00,2273.0,DONNA,1960.0
1,118,1961,Region VI,6,TX,Texas,Statewide,9/16/61,DR,Hurricane,...,9/16/61,5/4/64,48000.0,48,NaN,NaN,1961-09-16 18:00:00,2329.0,CARLA,1961.0
2,177,1964,Region IV,4,GA,Georgia,Statewide,9/10/64,DR,Hurricane,...,9/10/64,4/18/66,13000.0,13,NaN,NaN,1964-09-10 18:00:00,2549.0,DORA,1964.0
3,106,1960,Region IV,4,FL,Florida,Statewide,9/12/60,DR,Hurricane,...,9/12/60,5/24/62,12000.0,12,NaN,NaN,1960-09-12 18:00:00,2273.0,DONNA,1960.0
4,176,1964,Region IV,4,FL,Florida,Statewide,9/10/64,DR,Hurricane,...,9/10/64,8/8/66,12000.0,12,NaN,NaN,1964-09-10 18:00:00,2549.0,DORA,1964.0


In [48]:
public_assistance = pd.read_csv(
    'Data.gov_PublicAssistanceFundedProjectsDetails_9.12.2018.csv',
    skipinitialspace=True,
    skiprows=3)

public_assistance.columns = [c.strip() for c in public_assistance.columns]

In [49]:
# Assert there is exactly one ibtracs storm per disaster number
(disaster_summaries.groupby('Disaster Number').apply(lambda x: len(x.ibtracs_storm_id.unique())) == 1).all()

True

In [50]:
(
    len(disaster_summaries),
    len(disaster_summaries['Disaster Number'].drop_duplicates()),
    len(disaster_summaries[['Disaster Number', 'State', 'County']].drop_duplicates()))

(9207, 327, 9207)

In [52]:
disaster_numbers_for_storms = (
    disaster_summaries
    [['Disaster Number', 'ibtracs_storm_id', 'ibtracs_storm_name', 'ibtracs_storm_season']]
    .drop_duplicates()
    .set_index('Disaster Number'))

In [53]:
disaster_numbers_for_storms.head()

,ibtracs_storm_id,ibtracs_storm_name,ibtracs_storm_season
Disaster Number,,,
107,2273.0,DONNA,1960.0
118,2329.0,CARLA,1961.0
177,2549.0,DORA,1964.0
106,2273.0,DONNA,1960.0
176,2549.0,DORA,1964.0


In [54]:
public_assistance.head()

,Disaster Number,PW Number,Application Title,Applicant ID,Damage Category Code,Project Size,County,County Code,State,State Code,State Number Code,Project Amount,Federal Share Obligated,Total Obligated
0,4371,2,0047295 - NH-DOT-Cat. B Protective Measures Sn...,000-UZDQG-00,B - Protective Measures,Large,Statewide,0.0,New Hampshire,NH,33,"$795,600.74","$596,700.56","$596,700.56"
1,4371,3,0047334 - Snow Removal-Town of Derry,015-17940-00,B - Protective Measures,Large,Rockingham,15.0,New Hampshire,NH,33,"$144,080.23","$108,060.17","$108,060.17"
2,4371,1,0046576 - Snow removal and EOC Activation - SE...,015-33460-00,B - Protective Measures,Small,Rockingham,15.0,New Hampshire,NH,33,"$45,156.33","$33,867.25","$33,867.25"
3,4368,5,47685 - Butler Borough Snow Removal/EPM - Self...,027-09040-00,B - Protective Measures,Small,Morris,27.0,New Jersey,NJ,34,"$51,958.45","$38,968.84","$38,968.84"
4,4368,4,47641 - Washington_Snow Removal_EPM_Self Cert...,027-77240-00,B - Protective Measures,Small,Morris,27.0,New Jersey,NJ,34,"$51,905.91","$38,929.43","$38,929.43"


In [68]:
for col in ['Project Amount', 'Federal Share Obligated', 'Total Obligated']:
    public_assistance[col] = (
        public_assistance[col]
        .str.replace('\$','')
        .str.replace(',', '')
        .str.replace(r'-\s*', '0')
        .str.replace(r'[\(\)]', '')
        .astype(float))

In [69]:
public_assistance_for_named_hurricanes = public_assistance[
    np.isin(public_assistance['Disaster Number'], disaster_summaries['Disaster Number'])]

In [70]:
for col in disaster_numbers_for_storms.columns:
    if col in public_assistance_for_named_hurricanes.columns:
        continue
    
    public_assistance_for_named_hurricanes[col] = disaster_numbers_for_storms.reindex(
        public_assistance_for_named_hurricanes['Disaster Number'].values)[col].values

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [71]:
public_assistance_for_named_hurricanes.head()

,Disaster Number,PW Number,Application Title,Applicant ID,Damage Category Code,Project Size,County,County Code,State,State Code,State Number Code,Project Amount,Federal Share Obligated,Total Obligated,ibtracs_storm_id,ibtracs_storm_name,ibtracs_storm_season
15466,4291,11,JRKC004 - Holly Run Road,000-U2Q9Q-00,C - Roads and Bridges,Large,Statewide,0.0,Virginia,VA,51,178735.83,134051.87,134051.87,7340.0,MATTHEW,2016.0
15467,4291,6,RHHB001 - Emergency Protective Measures,000-UG67T-00,B - Protective Measures,Small,Statewide,0.0,Virginia,VA,51,4358.42,3268.82,3268.82,7340.0,MATTHEW,2016.0
15468,4291,15,TABE002 - Police Vehicle Damages,000-U0SVV-00,E - Public Buildings,Small,Statewide,0.0,Virginia,VA,51,8841.75,6631.31,6631.31,7340.0,MATTHEW,2016.0
15469,4291,14,WWKE017 - Fleet Vehicles,000-UZE2H-00,E - Public Buildings,Small,Statewide,0.0,Virginia,VA,51,71050.00,53287.50,53287.50,7340.0,MATTHEW,2016.0
15470,4291,18,LJDF006 - Electrical Line,000-UOZ67-00,F - Public Utilities,Small,Statewide,0.0,Virginia,VA,51,15750.54,11812.91,11812.91,7340.0,MATTHEW,2016.0


In [72]:
public_assistance_for_named_hurricanes.columns

Index(['Disaster Number', 'PW Number', 'Application Title', 'Applicant ID',
       'Damage Category Code', 'Project Size', 'County', 'County Code',
       'State', 'State Code', 'State Number Code', 'Project Amount',
       'Federal Share Obligated', 'Total Obligated', 'ibtracs_storm_id',
       'ibtracs_storm_name', 'ibtracs_storm_season'],
      dtype='object')

In [74]:
groupby_cols = [
    'State',
    'State Code',
    'State Number Code',
    'County',
    'County Code',
    'ibtracs_storm_id',
    'ibtracs_storm_name',
    'ibtracs_storm_season']

value_cols = ['Project Amount', 'Federal Share Obligated', 'Total Obligated']

aggregated_public_assistance_data = (
    public_assistance_for_named_hurricanes[groupby_cols + value_cols]
    .groupby(groupby_cols)
    .sum())

In [75]:
aggregated_public_assistance_data.head()

Project Amount  \
State   State Code State Number Code County  County Code ibtracs_storm_id ibtracs_storm_name ibtracs_storm_season                   
Alabama AL         1                 Autauga 1.0         6229.0           IVAN               2004.0                     197929.47   
                                                         6300.0           DENNIS             2005.0                      62089.50   
                                                         6600.0           GUSTAV             2008.0                       1219.55   
                                                         6657.0           LIN                2009.0                     432339.82   
                                     Baldwin 3.0         6047.0           ISIDORE            2002.0                    2029732.78   

                                                                                                                   Federal Share Obligated  \
State   State Code State Number Code County  County Code ibtracs_storm_id ibtracs_storm_name ibtracs_storm_season                            
Alabama AL         1                 Autauga 1.0         6229.0           IVAN               2004.0                              183683.18   
                                                         6300.0           DENNIS             2005.0                               62089.50   
                                                         6600.0           GUSTAV             2008.0                                 914.66   
                                                         6657.0           LIN                2009.0                              324254.89   
                                     Baldwin 3.0         6047.0           ISIDORE            2002.0                             1522299.61   

                                                                                                                   Total Obligated  
State   State Code State Number Code County  County Code ibtracs_storm_id ibtracs_storm_name ibtracs_storm_season                   
Alabama AL         1                 Autauga 1.0         6229.0           IVAN               2004.0                      190186.43  
                                                         6300.0           DENNIS             2005.0                       64473.73  
                                                         6600.0           GUSTAV             2008.0                         914.66  
                                                         6657.0           LIN                2009.0                      324254.89  
                                     Baldwin 3.0         6047.0           ISIDORE            2002.0                     1592610.47

In [76]:
aggregated_public_assistance_data.sum()

Project Amount             5.172421e+10
Federal Share Obligated    4.782328e+10
Total Obligated            4.813568e+10
dtype: float64

In [77]:
aggregated_public_assistance_data.to_csv(
    'Data.gov_PublicAssistanceFundedProjectsDetails_9.12.2018.aggregated_by_hurricane.csv')